In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv", nrows = 1000000)
test = pd.read_csv("../input/test.csv")

**PART 1 --> DATA CLEANSING & EXPLORATORY DATA ANALYSIS (EDA)**

Will perform the following activities
* Shape of train and test sets
* Check for NaNs and drop them (if any)
* Check for outliers and drop them (if any)
* Type conversion of relevant fields
* We Can do it easily with the help of pandas profiling.

In [ ]:
train.shape


In [ ]:
test.shape

In [ ]:
train.head(10)

In [ ]:
train.describe()

In [ ]:
#check for missing values in train data
train.isnull().sum().sort_values(ascending=False)

In [ ]:
#drop the missing values
train = train.drop(train[train.isnull().any(1)].index, axis = 0)

In [ ]:
train.shape

**Univariate Analysis**
* Now We Perform Univariate Analysis,i.e. Analysing Each variable one by one. To get best insights from it.

In [ ]:
#check the target column
train['fare_amount'].describe()

Fare amount has a negative value, which doesn't make sense. Remove these fields

In [ ]:
#38 fields have negative fare_amount values.
from collections import Counter
Counter(train['fare_amount']<0)

In [ ]:
train = train.drop(train[train['fare_amount']<0].index, axis=0)
train.shape

In [ ]:
#no more negative values in the fare field
train['fare_amount'].describe()

In [ ]:
#highest fare is $500
train['fare_amount'].sort_values(ascending=False)

In [ ]:
train['passenger_count'].describe()

In [ ]:
#max is 208 passengers. Assuming that a bus is a 'taxi' in NYC, I don't think a bus can carry 208 this is DEFINITELY an outlier. 
#Lets drop it 
train[train['passenger_count']>8]

In [ ]:
train = train.drop(train[train['passenger_count']==208].index, axis = 0)

In [ ]:
#much neater now! Max number of passengers are 6. Which makes sense is the cab is an SUV :)
train['passenger_count'].describe()

In [ ]:
#Next, let us explore the pickup latitude and longitudes
train['pickup_latitude'].describe()

Quick Googling gave me this info
* Latitudes range from -90 to 90.
* Longitudes range from -180 to 180.

The above describe clearly shows some outliers. Let's filter them

In [ ]:
train[train['pickup_latitude']<-90]

In [ ]:
train[train['pickup_latitude']>90]

In [ ]:
#We need to drop these outliers
train = train.drop(((train[train['pickup_latitude']<-90])|(train[train['pickup_latitude']>90])).index, axis=0)

In [ ]:
train.shape

In [ ]:
#similar operation for pickup longitude
train['pickup_longitude'].describe()

In [ ]:
train[train['pickup_longitude']<-180]

In [ ]:
train[train['pickup_longitude']>180]

In [ ]:
train = train.drop(((train[train['pickup_longitude']<-180])|(train[train['pickup_longitude']>180])).index, axis=0)

In [ ]:
#similar operation for dropoff latitude and longitude
train[train['dropoff_latitude']<-90]

In [ ]:
train[train['dropoff_latitude']>90]

In [ ]:
train = train.drop(((train[train['dropoff_latitude']<-90])|(train[train['dropoff_latitude']>90])).index, axis=0)

In [ ]:
train[train['dropoff_latitude']<-180]|train[train['dropoff_latitude']>180]

In [ ]:
train.dtypes

In [ ]:
train['key'] = pd.to_datetime(train['key'],infer_datetime_format = True)
train['pickup_datetime']  =  pd.to_datetime(train['pickup_datetime'],infer_datetime_format=True)

In [ ]:
#Convert for test data
test['key'] = pd.to_datetime(test['key'],infer_datetime_format = True)
test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'],infer_datetime_format = True)

In [ ]:
#check the dtypes after conversion
train.dtypes


First, let's split the datetime field 'pickup_datetime' to the following - 
* year
* month
* date
* hour
* day of week

Using these we shall calculate the day of the week and come to our conclusions about how pickup_location affects the fare.
Also, create a new field 'distance' to fetch the distance between the pickup and the drop.

We can calulate the distance in a sphere when latitudes and longitudes are given by [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula)

**haversine(θ) = sin²(θ/2)**

Eventually, the formual boils down to the following where φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km) to include latitude and longitude coordinates (A and B in this case).

**a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)**

**c = 2 * atan2( √a, √(1−a) )**

**d = R ⋅ c**

**d = Haversine distance**

*Refer [this](https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula) page for more info and examples on Haversine formula*

In [ ]:
def haversine_distance(lat1,long1, lat2,long2):
    data = [train,test]
    for i in data:
        r = 6371
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
        
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
        
        a = np.sin(delta_phi /2.0)**2 + np.cos(phi1)* np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        
        d = (r * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
train['H_Distance'].head(10)

In [ ]:
data = [train,test]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

There are values which are greater than 100 kms! In NYC I am not sure why people would take cabs to travel more than a 100 kms. Since the number of bins for 100-200 kms is quite high, I will keep these. These outliers could be because of typos or missing values in the latitude or longitude. Remove fields of the following - 
1.  Pickup latitude and pickup longitude are 0 but dropoff latitude and longitude are not 0, but the fare is 0
2. vice versa of point 1.
3. Pickup latitude and pickup longitude are 0 but dropoff latitude and longitude are not 0, but the fare is NOT 0. Here I will have to impute the distance values in both the train and test data.

In [ ]:
#pickup latitude and longitude = 0
train.loc[((train['pickup_latitude']==0) & (train['pickup_longitude']==0))&((train['dropoff_latitude']!=0) & (train['dropoff_longitude']!=0)) & (train['fare_amount']==0)]

In [ ]:
train = train.drop(train.loc[((train['pickup_latitude']==0) & (train['pickup_longitude']==0))&((train['dropoff_latitude']!=0) & (train['dropoff_longitude']!=0)) & (train['fare_amount']==0)].index, axis=0)

In [ ]:
#1 row dropped
train.shape

In [ ]:
#dropoff latitude and longitude = 0
train.loc[((train['pickup_latitude']!=0) & (train['pickup_longitude']!=0))&((train['dropoff_latitude']==0) & (train['dropoff_longitude']==0)) & (train['fare_amount']==0)]

In [ ]:
train = train.drop(train.loc[((train['pickup_latitude']!=0) & (train['pickup_longitude']!=0))&((train['dropoff_latitude']==0) & (train['dropoff_longitude']==0)) & (train['fare_amount']==0)].index, axis=0)

Check the H_Distance fields which are greater than 200 kms cause there is no way that people would travel more than 200 kms at the most in NYC in a CAB!

In [ ]:
high_distance = train.loc[(train['H_Distance']>200)&(train['fare_amount']!=0)]

In [ ]:
high_distance

In [ ]:
high_distance['H_Distance'] = high_distance.apply(
    lambda row: (row['fare_amount'] - 2.50)/1.56,
    axis=1
)

In [ ]:
#sync the train data with the newly computed distance values from high_distance dataframe
train.update(high_distance)

In [ ]:
train[train['H_Distance']==0]

We can see a few rows with distance =0. This could be due to 2 reasons 
1. The cab waited the whole time and the passenger eventually cancelled. *That's why the pickup and drop co-ordinates are the same and maybe, the passenger was charged for the waiting time.*
2. The pickup and drop co-ordinates were not entered. In other words, these are **missing values**!

28667 rows are too many rows to be deleted. We need to impute these missing values. I have a plan. I intend to impute the missing distance values with the fare and average price per kilometer of NYC cabs.

A quick Google search gave me the following prices  - 

* $$2.5 base-price  +  $1.56/km --> 6AM to 8PM Mon-Fri

* $$3.0 base-price  +  $1.56/km --> 8PM to 6AM Mon-Fri and Sat&Sun

However, before we proceed with the above steps, lets check for the following scenarios to impute the missing fare amount and the H_Distance in train data.



In [ ]:
train[(train['H_Distance']==0)&(train['fare_amount']==0)]

In [ ]:
train = train.drop(train[(train['H_Distance']==0)&(train['fare_amount']==0)].index, axis = 0)

In [ ]:
#4 rows dropped
train[(train['H_Distance']==0)].shape

In [ ]:
#Between 6AM and 8PM on Mon-Fri
rush_hour = train.loc[(((train['Hour']>=6)&(train['Hour']<=20)) & ((train['Day of Week']>=1) & (train['Day of Week']<=5)) & (train['H_Distance']==0) & (train['fare_amount'] < 2.5))]
rush_hour

In [ ]:
train=train.drop(rush_hour.index, axis=0)

In [ ]:
#Between 8PM and 6AM on Mon-Fri
non_rush_hour = train.loc[(((train['Hour']<6)|(train['Hour']>20)) & ((train['Day of Week']>=1)&(train['Day of Week']<=5)) & (train['H_Distance']==0) & (train['fare_amount'] < 3.0))]
#print(Counter(non_work_hours['Hour']))
#print(Counter(non_work_hours['Day of Week']))
non_rush_hour
#keep these. Since the fare_amount is not <2.5 (which is the base fare), these values seem legit to me.

In [ ]:
#Saturday and Sunday all hours
weekends = train.loc[((train['Day of Week']==0) | (train['Day of Week']==6)) & (train['H_Distance']==0) & (train['fare_amount'] < 3.0)]
weekends
#Counter(weekends['Day of Week'])
#keep these too. Since the fare_amount is not <2.5, these values seem legit to me.

In [ ]:
train.loc[(train['H_Distance']!=0) & (train['fare_amount']==0)]

In [ ]:
scenario_3 = train.loc[(train['H_Distance']!=0) & (train['fare_amount']==0)]

In [ ]:
len(scenario_3)

In [ ]:
#We do not have any distance values that are outliers.
scenario_3.sort_values('H_Distance', ascending=False)

In [ ]:
scenario_3['fare_amount'] = scenario_3.apply(
    lambda row: ((row['H_Distance'] * 1.56) + 2.50), axis=1
)

In [ ]:
scenario_3['fare_amount']

In [ ]:
train.update(scenario_3)

In [ ]:
train.loc[(train['H_Distance']==0) & (train['fare_amount']!=0)]

In [ ]:
scenario_4 = train.loc[(train['H_Distance']==0) & (train['fare_amount']!=0)]

In [ ]:
len(scenario_4)

In [ ]:
#Using our prior knowledge about the base price during weekdays and weekends for the cabs.
#I do not want to impute these 1502 values as they are legible ones.
scenario_4.loc[(scenario_4['fare_amount']<=3.0)&(scenario_4['H_Distance']==0)]

In [ ]:
scenario_4.loc[(scenario_4['fare_amount']>3.0)&(scenario_4['H_Distance']==0)]

In [ ]:
scenario_4_sub = scenario_4.loc[(scenario_4['fare_amount']>3.0)&(scenario_4['H_Distance']==0)]

In [ ]:
scenario_4_sub = scenario_4.loc[(scenario_4['fare_amount']>3.0)&(scenario_4['H_Distance']==0)]

In [ ]:
len(scenario_4_sub)

In [ ]:
scenario_4_sub['H_Distance'] = scenario_4_sub.apply(
lambda row: ((row['fare_amount']-2.50)/1.56), axis=1
)

In [ ]:
train.update(scenario_4_sub)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
#not including the pickup_datetime columns as datetime columns cannot be directly used while modelling. Features need to extracted from the 
#timestamp fields which will later be used as features for modelling.
train = train.drop(['key','pickup_datetime'], axis = 1)
test = test.drop(['key','pickup_datetime'], axis = 1)

In [ ]:
x_train = train.iloc[:,train.columns!='fare_amount']
y_train = train['fare_amount'].values
x_test = test

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
rf_predict = rf.predict(x_test)
#print(rf_predict)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['fare_amount'] = rf_predict
submission.to_csv('submission_1.csv', index=False)
submission.head(20)

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['fare_amount'] = lr_predict
submission.to_csv('submission_2.csv', index=False)
submission.head(20)